In [132]:
# Initial imports.
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [133]:
# Loading data after database join of seasons/quarters file and Apple's stock file
# https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e#:~:text=By%20default%2C%20date%20columns%20are%20parsed%20using%20the,a%20different%20date%20format%2C%20for%20example%2C%20YYYY-DD-MM%20HH%3AMM%3ASS%3A
file_path =Path("CLEAN_DATE_AAPL_Official_DB.csv")
apple_df = pd.read_csv(file_path, parse_dates=['Date'])
apple_df.set_index('Unnamed: 0',inplace=True)
apple_df.index.name=None
apple_df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Open_Change,High_Change,Low_Change,Close_Change,Volume_Change,Gain_Loss_Open,Gain_Loss_High,Gain_Loss_Low,Gain_Loss_Close,Gain_Loss_Volume
0.0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0,0.128348,0.128906,0.128348,0.128348,0,Gain,Gain,Gain,Gain,Gain
1.0,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0,-0.006138,-0.006696,-0.006696,-0.006696,-293148800,Loss,Loss,Loss,Loss,Loss
2.0,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0,-0.008929,-0.008929,-0.008929,-0.008929,-70156800,Loss,Loss,Loss,Loss,Loss
3.0,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0,0.002232,0.00279,0.00279,0.00279,-19286400,Gain,Gain,Gain,Gain,Loss
4.0,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0,0.003349,0.003349,0.003349,0.003349,-12992000,Gain,Gain,Gain,Gain,Loss


In [134]:
apple_df['Date'] = pd.to_datetime(apple_df['Date'])
apple_df['Date'] = apple_df['Date'].map(dt.datetime.toordinal)
apple_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Open_Change,High_Change,Low_Change,Close_Change,Volume_Change,Gain_Loss_Open,Gain_Loss_High,Gain_Loss_Low,Gain_Loss_Close,Gain_Loss_Volume
0.0,723161,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0,0.128348,0.128906,0.128348,0.128348,0,Gain,Gain,Gain,Gain,Gain
1.0,723164,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0,-0.006138,-0.006696,-0.006696,-0.006696,-293148800,Loss,Loss,Loss,Loss,Loss
2.0,723165,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0,-0.008929,-0.008929,-0.008929,-0.008929,-70156800,Loss,Loss,Loss,Loss,Loss
3.0,723166,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0,0.002232,0.00279,0.00279,0.00279,-19286400,Gain,Gain,Gain,Gain,Loss
4.0,723167,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0,0.003349,0.003349,0.003349,0.003349,-12992000,Gain,Gain,Gain,Gain,Loss


In [135]:
apple_adj_df = apple_df.drop(['High_Change','Low_Change','Close_Change','Volume_Change','Gain_Loss_High','Gain_Loss_Low','Gain_Loss_Close','Gain_Loss_Volume'], axis=1)
apple_adj_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Open_Change,Gain_Loss_Open
0.0,723161,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0,0.128348,Gain
1.0,723164,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0,-0.006138,Loss
2.0,723165,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0,-0.008929,Loss
3.0,723166,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0,0.002232,Gain
4.0,723167,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0,0.003349,Gain


In [136]:
apple_adj_df.dtypes

Date                int64
Open              float64
High              float64
Low               float64
Close             float64
Adj Close         float64
Volume            float64
Open_Change        object
Gain_Loss_Open     object
dtype: object

In [137]:
apple_adj_df["Open_Change"] = pd.to_numeric(apple_adj_df.Open_Change,errors='coerce')

In [138]:
apple_adj_df.dtypes

Date                int64
Open              float64
High              float64
Low               float64
Close             float64
Adj Close         float64
Volume            float64
Open_Change       float64
Gain_Loss_Open     object
dtype: object

In [139]:
apple_adj_df.count()

Date              10227
Open              10226
High              10226
Low               10226
Close             10226
Adj Close         10226
Volume            10226
Open_Change       10226
Gain_Loss_Open    10227
dtype: int64

In [151]:
apple_adj_df.isnull().sum()

Date              0
Open              1
High              1
Low               1
Close             1
Adj Close         1
Volume            1
Open_Change       1
Gain_Loss_Open    0
dtype: int64

In [126]:
#apple_adj_df.dtypes

Date                int64
Open              float64
High              float64
Low               float64
Close             float64
Adj Close         float64
Volume            float64
Open_Change       float64
Gain_Loss_Open     object
dtype: object

In [152]:
apple_adj_df["Open"] = apple_adj_df["Open"].fillna(0)

In [154]:
apple_adj_df["High"] = apple_adj_df["High"].fillna(0)

In [155]:
apple_adj_df["Low"] = apple_adj_df["Low"].fillna(0)

In [156]:
apple_adj_df["Close"] = apple_adj_df["Close"].fillna(0)

In [157]:
apple_adj_df["Adj Close"] = apple_adj_df["Adj Close"].fillna(0)

In [158]:
apple_adj_df["Volume"] = apple_adj_df["Volume"].fillna(0)

In [159]:
apple_adj_df["Open_Change"] = apple_adj_df["Open_Change"].fillna(0)

In [160]:
apple_adj_df.isnull().sum()

Date              0
Open              0
High              0
Low               0
Close             0
Adj Close         0
Volume            0
Open_Change       0
Gain_Loss_Open    0
dtype: int64

In [161]:
apple_adj_df.count()

Date              10227
Open              10227
High              10227
Low               10227
Close             10227
Adj Close         10227
Volume            10227
Open_Change       10227
Gain_Loss_Open    10227
dtype: int64

In [176]:
apple_adj_df.to_csv('CLEAN_DATE_AAPL_No_NaN.csv')

In [11]:
# np.any(np.isnan(mat))
# np.all(np.isfinite(mat))

#With np.isnan(X) you get a boolean mask back with True for positions containing NaNs.

#With np.where(np.isnan(X)) you get back a tuple with i, j coordinates of NaNs.

#Finally, with np.nan_to_num(X) you "replace nan with zero and inf with finite numbers".

In [162]:
# Must remain in order to activate label_binarize
#If in doubt refer to sklearn
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
apple_adj_df = apple_adj_df.copy()
apple_adj_df['Gain_Loss_Open'] = le.fit_transform(apple_adj_df['Gain_Loss_Open'])
apple_adj_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Open_Change,Gain_Loss_Open
0.0,723161,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0,0.128348,0
1.0,723164,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0,-0.006138,1
2.0,723165,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0,-0.008929,1
3.0,723166,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0,0.002232,0
4.0,723167,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0,0.003349,0


In [163]:
# If in doubt look up scikit learn label_binarize
from sklearn.preprocessing import label_binarize

binarized_gain_loss = label_binarize(y=list(apple_adj_df['Gain_Loss_Open']), classes=[1,0])
apple_adj_df = apple_adj_df.copy()
apple_adj_df['Gain_Loss_Open'] = binarized_gain_loss
apple_adj_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Open_Change,Gain_Loss_Open
0.0,723161,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0,0.128348,1
1.0,723164,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0,-0.006138,0
2.0,723165,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0,-0.008929,0
3.0,723166,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0,0.002232,1
4.0,723167,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0,0.003349,1


In [164]:
# Define features set
X = apple_adj_df.copy()
X = apple_adj_df.drop(["Gain_Loss_Open","Open_Change"], axis=1)
X.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0.0,723161,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0
1.0,723164,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0
2.0,723165,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0
3.0,723166,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0
4.0,723167,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0


In [143]:
#np.isnan(X)

,Date,Open,High,Low,Close,Adj Close,Volume
0.0,False,False,False,False,False,False,False
1.0,False,False,False,False,False,False,False
2.0,False,False,False,False,False,False,False
3.0,False,False,False,False,False,False,False
4.0,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
10222.0,False,False,False,False,False,False,False
10223.0,False,False,False,False,False,False,False
10224.0,False,False,False,False,False,False,False
10225.0,False,False,False,False,False,False,False


In [144]:
#np.where(np.isnan(X))

(array([10226, 10226, 10226, 10226, 10226, 10226], dtype=int64),
 array([1, 2, 3, 4, 5, 6], dtype=int64))

In [145]:
#np.nan_to_num(X)

array([[7.23161000e+05, 1.28348000e-01, 1.28906000e-01, ...,
        1.28348000e-01, 1.00751000e-01, 4.69033600e+08],
       [7.23164000e+05, 1.22210000e-01, 1.22210000e-01, ...,
        1.21652000e-01, 9.54950000e-02, 1.75884800e+08],
       [7.23165000e+05, 1.13281000e-01, 1.13281000e-01, ...,
        1.12723000e-01, 8.84850000e-02, 1.05728000e+08],
       ...,
       [7.38128000e+05, 1.32520004e+02, 1.32850006e+02, ...,
        1.31240005e+02, 1.31017456e+02, 9.14200000e+07],
       [7.38129000e+05, 1.23400002e+02, 1.24639999e+02, ...,
        1.22769997e+02, 1.22769997e+02, 1.12172300e+08],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [165]:
# Define the target set.
y = apple_adj_df["Gain_Loss_Open"].ravel()
y[:5]

array([1, 0, 0, 1, 1])

In [166]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [167]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [168]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=100, random_state=78) 

In [169]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [170]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [171]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,574,602
Actual 1,479,902


In [172]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [173]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,574,602
Actual 1,479,902


Accuracy Score : 0.577238951896754
Classification Report
              precision    recall  f1-score   support

           0       0.55      0.49      0.52      1176
           1       0.60      0.65      0.63      1381

    accuracy                           0.58      2557
   macro avg       0.57      0.57      0.57      2557
weighted avg       0.57      0.58      0.57      2557



In [174]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.18241305, 0.13378675, 0.12965709, 0.13183945, 0.12662915,
       0.13256455, 0.16310996])

In [175]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.18241305222448534, 'Date'),
 (0.16310995508712725, 'Volume'),
 (0.13378675017075958, 'Open'),
 (0.1325645542623745, 'Adj Close'),
 (0.13183945135826688, 'Low'),
 (0.12965709117813587, 'High'),
 (0.1266291457188507, 'Close')]